# File: Import analysis
Project: Trade war on labor market\
Author: Xing Xu\
Created in Nov. 2021\
Description: Import tariff on US trade and labor market\
This file processes the raw data of QCEW (https://www.bls.gov/cew/downloadable-data-files.htm) from BLS, the import data from Census Bureau and the tariff data from Bown et al. (2019), then get to the county level using a simplified import penetration assumption, then we look at the county level import change throughout the US-China trade war \
Note that I use QCEW by industry at NAICS4 level and the tariff rate is at a HS10 level. With a concordance table from census bureau and additional information from Justin Pierce and Peter Schott (2009), HS10 commodities can be assigned to different NAICS4 industries. Then a weighted average is taken at each NAICS4 level with regard to the relative import value from the world of each commodity at the level.\
\
Abbreviations: 
- HS10: 10 digit Harmonized System code (Commodity level)
- NAICS4: 4 digit North American Industry Classification System (Industry level)
- BLS: U.S. Bureau of Labor Statistics

In [1]:
# Packages you will need
import pandas as pd
import numpy as np
import datetime as dt
import requests, io
import glob
import json
import geopandas
from shapely.geometry import box

In [2]:
# Set directory, change to reproduce the results
import os
os.chdir(r'C:\Users\2xu2\International_trade\data')
# path is for loading QCEW data in bunch
path = r'C:\Users\2xu2\International_trade\data\QCEW_NAICS4'

## Import data
Get HS10 Chinese import data from Census Bureau via API.

In [3]:
my_key = "&key=3c7e749a86a0c6eba2e72bbef69a1740883f4b2f"
# This is my key for Census Bureau API. 
# If you want to do more with it, please get your own key.

end_use = "hs?get=I_COMMODITY,CTY_CODE,CTY_NAME,GEN_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time=from+2016-01" + "&COMM_LVL=HS10"
# Specify country, 5700 is the country code of China
url = url + "&CTY_CODE=5700"

r = requests.get(url) 

# Convert to dataframe
HS10importdata = pd.DataFrame(r.json()[1:]) 

HS10importdata.columns = r.json()[0]
HS10importdata.time = pd.to_datetime(HS10importdata.time, 
                                     format="%Y-%m")

HS10importdata["china_import"] = HS10importdata.GEN_VAL_MO.astype(float)
HS10importdata.I_COMMODITY = HS10importdata.I_COMMODITY.astype(str)
HS10importdata = HS10importdata[HS10importdata['time'] <= pd.Timestamp(2021, 2, 1)]
HS10importdata = HS10importdata.drop(['CTY_CODE', 'CTY_NAME', 'COMM_LVL', 'GEN_VAL_MO'], axis = 1)
HS10importdata.head()

,I_COMMODITY,time,china_import
0,9403905080,2016-01-01,1649607.0
1,9401104000,2016-01-01,52036.0
2,9401108000,2016-01-01,84315.0
3,9401200000,2016-01-01,4087729.0
4,9401304000,2016-01-01,1514301.0


In [4]:
# base time total import (2017)
HS10importdata2017 = HS10importdata[(HS10importdata['time'] >= pd.Timestamp(2017,1,1)) & 
                                (HS10importdata['time'] <= pd.Timestamp(2017,12,1))]
HS10importdata2017 = HS10importdata2017.groupby('I_COMMODITY')['china_import'].sum()
HS10importdata2017.head()

I_COMMODITY
0101290090     1169976.0
0106110000    33808661.0
0106199120        3000.0
0106199195       94733.0
0106200000       13358.0
Name: china_import, dtype: float64

## Tariff data
China import tariff data from Bown et al. (2021)

In [5]:
importtariff = pd.read_excel('2.Tariff Data Raw.xlsx', sheet_name = 'US')
# Let's just use the 2018 MFN rate for simplicity
# We don't care about the 2016 and 2017 tariff and there was only minor regular yearly adjustment
# The original data has all columns with strings, we have to clean it by hand
importtariff.columns = ['HS10', 'Description', '2016-01-01', '2018-02-07', '2018-03-23', 
                        '2018-07-06', '2018-08-23', '2018-09-24', '2019-02-07',
                        '2019-06-01', '2019-09-01', '2020-02-07', '2020-02-08',
                        '2020-02-14', '2017CHNimport']
importtariff = importtariff.drop(['Description', '2017CHNimport'], axis = 1).set_index(['HS10']).fillna(0)

# Note the numbers are changes in tariff
# Use iteration to get tariff at each time
dates = ['2016-01-01', '2018-02-07', '2018-03-23', '2018-07-06', 
         '2018-08-23', '2018-09-24', '2019-02-07', '2019-06-01', 
         '2019-09-01', '2020-02-07', '2020-02-08','2020-02-14']
for i in importtariff.index:
    for j in range(1,12):
        importtariff.loc[i, dates[j]] = importtariff.loc[i, dates[j - 1]] + importtariff.loc[i, dates[j]]
importtariff.head()

,2016-01-01,2018-02-07,2018-03-23,2018-07-06,2018-08-23,2018-09-24,2019-02-07,2019-06-01,2019-09-01,2020-02-07,2020-02-08,2020-02-14
HS10,,,,,,,,,,,,
101210010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,15.0,7.5
101210020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,15.0,7.5
101290090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,15.0,7.5
101904000,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,19.5,19.5,19.5,12.0
102210010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,15.0,7.5


In [6]:
# Convert wide to long now
importtariff = importtariff.stack()
# Sweep the floor after convert wide to long
importtariff = importtariff.reset_index().rename({'level_1':'date', 0:'UStariff'}, axis =1)

importtariff.date = pd.to_datetime(importtariff.date)
importtariff = importtariff.set_index('date')
importtariff.head()

,HS10,UStariff
date,,
2016-01-01,101210010,0.0
2018-02-07,101210010,0.0
2018-03-23,101210010,0.0
2018-07-06,101210010,0.0
2018-08-23,101210010,0.0


In [7]:
# Now, we assume tariff are enacted at the beginning of next consecutive month
# More discussion on this in the paper
datarange = pd.date_range('2016-01-01', '2020-02-01', freq='MS')

filled_importtariff = importtariff.groupby('HS10').apply(
    lambda x : x.reindex(datarange, 
                         method = "ffill"))
filled_importtariff = filled_importtariff.drop('HS10', axis = 1).reset_index()
filled_importtariff = filled_importtariff.rename({'level_1':'date'}, axis = 1)
filled_importtariff.head()

,HS10,date,UStariff
0,101210010,2016-01-01,0.0
1,101210010,2016-02-01,0.0
2,101210010,2016-03-01,0.0
3,101210010,2016-04-01,0.0
4,101210010,2016-05-01,0.0


In [8]:
# Note here we have to specify int64 due to the data is too long
HS10importdata.I_COMMODITY = HS10importdata.I_COMMODITY.astype('int64')
# Bring in HS10 import data
HS10_industry = pd.merge(filled_importtariff, HS10importdata,
                         how = "left", #imports might be zero due to reason illustrated in [data-preprocessing]
                         left_on = ['HS10', 'date'],
                         right_on = ['I_COMMODITY', 'time'])

# fillna(0) because missing values are 0 trade not recorded in Census Bureau database
HS10_industry = HS10_industry.drop(['I_COMMODITY','time'], axis = 1).fillna(0)
HS10_industry.head()

,HS10,date,UStariff,china_import
0,101210010,2016-01-01,0.0,0.0
1,101210010,2016-02-01,0.0,0.0
2,101210010,2016-03-01,0.0,0.0
3,101210010,2016-04-01,0.0,0.0
4,101210010,2016-05-01,0.0,0.0


In [9]:
# Use Census Bureau 2017 import concordance table to concord to NAICS4 level
importconcordance = pd.read_excel('impconcord17.xlsx')
importconcordance = importconcordance[['commodity', 'naics']]
# Get to NAICS4
importconcordance.naics = importconcordance.naics.str[:-2].astype(int)
# Merge with HS10 dataset
HS10_concord = pd.merge(HS10_industry, importconcordance,
                         how = "left",
                         left_on = "HS10",
                         right_on = 'commodity')
HS10_concord.tail()

,HS10,date,UStariff,china_import,commodity,naics
901345,9999950000,2019-10-01,0.0,344663090.0,9.999950e+09,9900.0
901346,9999950000,2019-11-01,0.0,330134811.0,9.999950e+09,9900.0
901347,9999950000,2019-12-01,0.0,326276074.0,9.999950e+09,9900.0
901348,9999950000,2020-01-01,0.0,326676778.0,9.999950e+09,9900.0
901349,9999950000,2020-02-01,0.0,234727076.0,9.999950e+09,9900.0


## QCEW data for employment
Use the BLS QCEW data at NAICS4 industry and county level from 2016 to 2020.\
It is called "quarterly" but it actually contains monthly employment data so that's what I am going to use.\
The whole dataset is awefully large (3.81 GB for only 4 years' NAICS4 data) so it is just not reasonable to use API.\
I will have to use downloaded dataset for the task.\
First use the industry level employment then go to county level.

In [10]:
# Now, bring employment data in
QCEW_files = glob.glob(path + "/*.csv")
temp = []

# You don't need to run this, just check the 'NAICS4_industry_import.csv', which is the output of the industry analysis
# If you do want to reproduce the result, use the downloaded version, this shall take 30 seconds to a minute

# Read the QCEW NAICS4 nationwise data from 2016 to 2020
for filename in QCEW_files:
    df = pd.read_csv(filename, index_col=None, header=0).tail(4)
    temp.append(df)
    
# Also get rid of some other issues with the data
QCEWUS = pd.concat(temp, axis=0, ignore_index=True)

C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# clean the data and convert wide to long
QCEWnation = QCEWUS[['industry_code', 'year', 'qtr', 
                     'month1_emplvl', 'month2_emplvl', 
                     'month3_emplvl']]
QCEWnation = QCEWnation.sort_values(['industry_code', 
                                     'year', 'qtr'])
QCEWnation = QCEWnation.set_index(['industry_code', 
                                   'year', 'qtr']).stack()
QCEWnation = QCEWnation.rename({'month1_emplvl':1, 
                                'month2_emplvl':2, 
                                'month3_emplvl':3})
QCEWnation = QCEWnation.reset_index()

# Get the datetime correct
QCEWnation['mon'] = 3*(QCEWnation['qtr'] - 1) + QCEWnation['level_3']
QCEWnation['date'] = pd.to_datetime(QCEWnation[['year',
                                                'mon']].astype(str).apply('-'.join,1))
QCEWnation = QCEWnation.drop(['qtr','level_3', 
                              'year', 'mon'], axis = 1)

# Employment level data by month
QCEWnation = QCEWnation.rename({0:'emplvl', 
                                'industry_code' : 'NAICS4'}, axis = 1)

QCEWnation = QCEWnation[QCEWnation['date'] <= pd.Timestamp(2020, 2, 1)]
QCEWnation.head()

,NAICS4,emplvl,date
0,1111,46879,2016-01-01
1,1111,47328,2016-02-01
2,1111,49197,2016-03-01
3,1111,53431,2016-04-01
4,1111,55389,2016-05-01


## HS10 to NAICS4

Now get the total import of US from Census Bureau via API\
Use the total import from the world as weight for the tariff impact on NAICS4 industries.

In [12]:
# get the total import from the world
my_key = "&key=3c7e749a86a0c6eba2e72bbef69a1740883f4b2f"
# This is my key for Census Bureau API. 
# If you want to do more with it, please get your own key.

end_use = "hs?get=I_COMMODITY,GEN_VAL_MO"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time=from+2016-01" + "&COMM_LVL=HS10"

r = requests.get(url) 

HS10totalimport = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
# Note that the first entry is the labels

HS10totalimport.columns = r.json()[0]
HS10totalimport.time = pd.to_datetime(HS10totalimport.time, 
                                     format="%Y-%m")

HS10totalimport["total_import"] = HS10totalimport.GEN_VAL_MO.astype(float)
HS10totalimport.I_COMMODITY = HS10totalimport.I_COMMODITY.astype(str)
HS10totalimport = HS10totalimport[HS10totalimport['time'] <= pd.Timestamp(2021, 2, 1)]
HS10totalimport = HS10totalimport.drop(['COMM_LVL', 'GEN_VAL_MO'], axis = 1)
HS10totalimport.head()

,I_COMMODITY,time,total_import
0,8541100040,2016-01-01,3927034.0
1,8541100050,2016-01-01,6071670.0
2,8541100060,2016-01-01,2054806.0
3,8541100070,2016-01-01,2576334.0
4,8541100080,2016-01-01,26729911.0


In [13]:
# merge the data back to our original data set
HS10totalimport.I_COMMODITY = HS10totalimport.I_COMMODITY.astype('int64')
HS10fullset = pd.merge(HS10_concord, HS10totalimport,
                      how = "left",
                      left_on = ['HS10', 'date'],
                      right_on = ['I_COMMODITY', 'time'])
HS10fullset = HS10fullset.rename({'naics':'NAICS4'}, axis = 1)
HS10fullset.head()

,HS10,date,UStariff,china_import,commodity,NAICS4,I_COMMODITY,time,total_import
0,101210010,2016-01-01,0.0,0.0,101210010.0,1129.0,101210010.0,2016-01-01,3812302.0
1,101210010,2016-02-01,0.0,0.0,101210010.0,1129.0,101210010.0,2016-02-01,2261550.0
2,101210010,2016-03-01,0.0,0.0,101210010.0,1129.0,101210010.0,2016-03-01,7676908.0
3,101210010,2016-04-01,0.0,0.0,101210010.0,1129.0,101210010.0,2016-04-01,2475003.0
4,101210010,2016-05-01,0.0,0.0,101210010.0,1129.0,101210010.0,2016-05-01,570486.0


In [14]:
# base level total import (2017)
# There are easier ways here but it is most intuitive to just get the subsample
HS10totalimport2017 = HS10fullset[(HS10fullset['time'] >= pd.Timestamp(2017,1,1)) & 
                                (HS10fullset['time'] <= pd.Timestamp(2017,12,1))]
HS10totalimport2017 = HS10totalimport2017.groupby('HS10')['total_import'].sum().reset_index()
# Now merge back to HS10 level
HS10totalimport2017 = pd.merge(HS10totalimport2017, 
                               HS10fullset[['HS10', 'NAICS4']].drop_duplicates(),
                              how = 'left',
                              left_on = 'HS10', right_on = 'HS10')
HS10totalimport2017.head()

,HS10,total_import,NAICS4
0,101210010,64492842.0,1129.0
1,101210020,56457171.0,1129.0
2,101290090,546454859.0,1129.0
3,101904000,6400.0,1129.0
4,102210010,244894.0,1121.0


In [15]:
# Merge back and get the weight
NAICS4totalimport2017 = HS10totalimport2017.groupby('NAICS4')['total_import'].sum()
NAICS4totalimport2017 = NAICS4totalimport2017.reset_index().rename({'total_import':'NAICS4total'}, axis = 1)
HS10totalimport2017 = pd.merge(HS10totalimport2017, NAICS4totalimport2017,
                               how = 'left',
                              left_on = 'NAICS4', right_on = 'NAICS4')

HS10totalimport2017['weight'] = HS10totalimport2017['total_import']/HS10totalimport2017['NAICS4total'] 
HS10totalimport2017.head()

,HS10,total_import,NAICS4,NAICS4total,weight
0,101210010,64492842.0,1129.0,1.511249e+09,0.042675
1,101210020,56457171.0,1129.0,1.511249e+09,0.037358
2,101290090,546454859.0,1129.0,1.511249e+09,0.361592
3,101904000,6400.0,1129.0,1.511249e+09,0.000004
4,102210010,244894.0,1121.0,1.576910e+09,0.000155


In [16]:
# Now get to the NAICS4 level by calculating the weighted tariff
NAICS4fullset = pd.merge(HS10fullset,HS10totalimport2017[['HS10', 'weight']],
                        how = "left",
                        left_on = "HS10", right_on = "HS10")
NAICS4fullset = NAICS4fullset.drop(['time', 'I_COMMODITY'], axis = 1)
NAICS4fullset['weighted_tariff'] = NAICS4fullset['UStariff'] * NAICS4fullset['weight']

# Get to NAICS4
NAICS4fullset = NAICS4fullset.drop(['HS10', 'UStariff', 'weight'], axis = 1)
NAICS4fullset = NAICS4fullset.groupby(['NAICS4', 'date']).sum().reset_index()
NAICS4fullset.head()

,NAICS4,date,china_import,commodity,total_import,weighted_tariff
0,1111.0,2016-01-01,9351822.0,1.190223e+11,271853483.0,0.501397
1,1111.0,2016-02-01,6817036.0,1.190223e+11,217027631.0,0.501397
2,1111.0,2016-03-01,7240204.0,1.190223e+11,301227559.0,0.501397
3,1111.0,2016-04-01,5832994.0,1.190223e+11,262957341.0,0.501397
4,1111.0,2016-05-01,6601558.0,1.190223e+11,213649478.0,0.501397


In [17]:
# Now we merge the two datasets
NAICS4fullset = pd.merge(NAICS4fullset, QCEWnation, 
                                  how = 'right',
                                  left_on = ['NAICS4', 'date'],
                                  right_on = ['NAICS4', 'date'])
# those with no trade, we give it 0
NAICS4fullset = NAICS4fullset.fillna(0)
# Give zero values to industries without trade
NAICS4fullset.head()

,NAICS4,date,china_import,commodity,total_import,weighted_tariff,emplvl
0,1111.0,2016-01-01,9351822.0,1.190223e+11,271853483.0,0.501397,46879
1,1111.0,2016-02-01,6817036.0,1.190223e+11,217027631.0,0.501397,47328
2,1111.0,2016-03-01,7240204.0,1.190223e+11,301227559.0,0.501397,49197
3,1111.0,2016-04-01,5832994.0,1.190223e+11,262957341.0,0.501397,53431
4,1111.0,2016-05-01,6601558.0,1.190223e+11,213649478.0,0.501397,55389


In [18]:
# Get the total import from China for each industry from 2016 to Feb, 2020
total_import_China = NAICS4fullset.groupby('NAICS4')['china_import'].sum()
total_import_China = total_import_China.reset_index().rename({'china_import':'total_china_import'}, axis = 1)
NAICS4fullset = pd.merge(NAICS4fullset, total_import_China,
                         how = "left",
                         left_on = "NAICS4",
                         right_on = "NAICS4")
NAICS4fullset.head()                       

,NAICS4,date,china_import,commodity,total_import,weighted_tariff,emplvl,total_china_import
0,1111.0,2016-01-01,9351822.0,1.190223e+11,271853483.0,0.501397,46879,268111471.0
1,1111.0,2016-02-01,6817036.0,1.190223e+11,217027631.0,0.501397,47328,268111471.0
2,1111.0,2016-03-01,7240204.0,1.190223e+11,301227559.0,0.501397,49197,268111471.0
3,1111.0,2016-04-01,5832994.0,1.190223e+11,262957341.0,0.501397,53431,268111471.0
4,1111.0,2016-05-01,6601558.0,1.190223e+11,213649478.0,0.501397,55389,268111471.0


In [19]:
# Get the NAICS2 code
NAICS4fullset['NAICS2'] = NAICS4fullset.NAICS4.astype(str).str[:-4].astype(int)
NAICS4fullset = NAICS4fullset.drop(['commodity'], axis = 1)
NAICS4fullset.head() 

,NAICS4,date,china_import,total_import,weighted_tariff,emplvl,total_china_import,NAICS2
0,1111.0,2016-01-01,9351822.0,271853483.0,0.501397,46879,268111471.0,11
1,1111.0,2016-02-01,6817036.0,217027631.0,0.501397,47328,268111471.0,11
2,1111.0,2016-03-01,7240204.0,301227559.0,0.501397,49197,268111471.0,11
3,1111.0,2016-04-01,5832994.0,262957341.0,0.501397,53431,268111471.0,11
4,1111.0,2016-05-01,6601558.0,213649478.0,0.501397,55389,268111471.0,11


Get the treatment indicator for each group, here we want to find the first period of tariff increase for each industry and assign 1 to all values from the period. The next block adopts a very clever way to do this.

In [20]:
# Thanks for Hu for giving me advice with using cumsum with groupby

# treatment time is defined as the first month each industry experience a tariff change
NAICS4fullset['pre_tariff'] = NAICS4fullset.groupby('NAICS4')['weighted_tariff'].shift()
NAICS4fullset['tariff_change'] = NAICS4fullset['weighted_tariff'] - NAICS4fullset['pre_tariff'] 
NAICS4fullset['treatment0'] = NAICS4fullset['tariff_change'].apply(lambda x: 1 if x > 0 else 0)

# This is actually a super cool and clean method
NAICS4fullset['treatment'] = NAICS4fullset.groupby('NAICS4')['treatment0'].transform(pd.Series.cumsum)
NAICS4fullset['treatment'] = NAICS4fullset['treatment'].apply(lambda x: 1 if x >= 1 else 0)
NAICS4fullset = NAICS4fullset.drop(['pre_tariff', 'tariff_change', 'treatment0'], axis = 1)
NAICS4fullset[NAICS4fullset['NAICS4'] == 1111].tail(20)

,NAICS4,date,china_import,total_import,weighted_tariff,emplvl,total_china_import,NAICS2,treatment
30,1111.0,2018-07-01,6186467.0,233679292.0,0.501397,58632,268111471.0,11,0
31,1111.0,2018-08-01,6548512.0,201570377.0,0.501397,58074,268111471.0,11,0
32,1111.0,2018-09-01,5210203.0,172751013.0,0.501397,58646,268111471.0,11,0
33,1111.0,2018-10-01,4829789.0,182351466.0,8.951857,62062,268111471.0,11,1
34,1111.0,2018-11-01,3244883.0,163503670.0,8.951857,57915,268111471.0,11,1
35,1111.0,2018-12-01,9631986.0,205960555.0,8.951857,54166,268111471.0,11,1
36,1111.0,2019-01-01,3225478.0,231010159.0,8.951857,47848,268111471.0,11,1
37,1111.0,2019-02-01,3209088.0,170733029.0,8.951857,47707,268111471.0,11,1
38,1111.0,2019-03-01,3025154.0,234043214.0,8.951857,49148,268111471.0,11,1
39,1111.0,2019-04-01,2934380.0,234798627.0,8.951857,52432,268111471.0,11,1


Awesome, we get it. Note that the weighted_tariff here is the average tariff percentage point, so NAICS4 has an average tariff of 0.5% tariff in Jan, 2016. Now I save it as a csv and use it for Stata regression analysis.

In [21]:
NAICS4fullset.to_csv('outputdata/NAICS4_industry_import.csv')

## County Import Tariff Exposure
Inspired by Autor, Dorn and Hansen (2017), here I adopt a similar but slightly simplified share-shift county exposure to import tariff:\
Formally the exposure to tariff on Chinese import of county c at time t is defined by:\
$ \phi_{c,t} = \sum \limits_{i\in I} \frac{E_{c,i, 2017}}{E_{c, 2017}} * \tau^{import}_{i, t}$\
where $\phi_{c,t}$ is the exposure to US import tariff on China, $E_{c, i, t}$ is the employment at county and industry level (2017 is the base period here), $\tau^{import}_{i, t}$ is the import tariff of US on China.\
\
This part is rather tedious. We merge the industry level data to county level by doing the estimation above.\
Code-wise, what we want to do is to get a weighted average of tariff by the employment level at each industry and county.\
Let's first get the base level average employment (note everything we do we keep it by month).\
\
**First thing first: I load all the NAICS4 county level QCEW data and get the base level employment**\
Note that the whole dataset is too big to efficiently use API, so I use the downloaded version.\
If you want to reproduce the result, download the data from (https://www.bls.gov/cew/downloadable-data-files.htm)

In [107]:
# Just a small notice that running this block takes a little while
# Should be 30 seconds - 1 minute depending on your hardware
# I am not a computer guy but I think the speed is related to your CPU and RAM.
# A warning should pop up which is OK, I get rid of the strings later
QCEW_files = glob.glob(path + "/*.csv")
temp = []

# Read all QCEW data from 2016 to 2020
for filename in QCEW_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    
    # keep only private sectors
    df = df[df['own_title'] == 'Private']
    
    # Get rid of the unwanted lines that don't contain county data
    df = df[~df.area_fips.str.contains("C", na=False)]
    df = df[~df.area_fips.str.contains("U", na=False)]
    df = df[~df.area_title.str.contains("Statewide", na=False)]
    
    # Select the variables that we need
    df = df[['area_fips', 'industry_code', 
                       'year', 'qtr', 'month1_emplvl', 
                       'month2_emplvl', 'month3_emplvl', 
                       'total_qtrly_wages', 'avg_wkly_wage']]
    df = df[~((df['year'] == 2020) & (df['qtr'] > 1))]
    temp.append(df)
    
frame = pd.concat(temp, axis=0, ignore_index=True)

#Get the type right
frame.area_fips = frame.area_fips.astype(int)

#drop the unclassified industry
frame = frame[frame['industry_code'] != 9999]

print(frame.shape)
frame.head(10)

C:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(7865018, 9)


,area_fips,industry_code,year,qtr,month1_emplvl,month2_emplvl,month3_emplvl,total_qtrly_wages,avg_wkly_wage
0,1001,1111,2016,1,0,0,0,0,0
1,1001,1111,2016,2,0,0,0,0,0
2,1001,1111,2016,3,0,0,0,0,0
3,1001,1111,2016,4,0,0,0,0,0
4,1003,1111,2016,1,0,0,0,0,0
5,1003,1111,2016,2,0,0,0,0,0
6,1003,1111,2016,3,0,0,0,0,0
7,1003,1111,2016,4,0,0,0,0,0
8,1033,1111,2016,1,0,0,0,0,0
9,1033,1111,2016,2,0,0,0,0,0


In [108]:
# This also take some time, there are 7.9 million rows of data
county_NAICS4 = frame.sort_values(['area_fips', 'industry_code', 
                                     'year', 'qtr'])

# Convert wide to long
wages_NAICS4 = county_NAICS4[['area_fips', 'industry_code', 'year', 'qtr', 'total_qtrly_wages', 'avg_wkly_wage']]
county_NAICS4 = county_NAICS4.drop(['total_qtrly_wages', 'avg_wkly_wage'], axis = 1)
county_NAICS4 = county_NAICS4.set_index(['area_fips', 'industry_code', 'year', 'qtr']).stack()
county_NAICS4 = county_NAICS4.rename({'month1_emplvl':1, 
                                'month2_emplvl':2, 
                                'month3_emplvl':3})

county_NAICS4 = county_NAICS4.reset_index()
county_NAICS4.head()

,area_fips,industry_code,year,qtr,level_4,0
0,1001,1111,2016,1,1,0
1,1001,1111,2016,1,2,0
2,1001,1111,2016,1,3,0
3,1001,1111,2016,2,1,0
4,1001,1111,2016,2,2,0


In [109]:
# This also take quite a while, again playing with a huge dataset is no fun
# Actually this takes forever, email me at xxu457@wisc.edu if you have a better solution
county_NAICS4["mon"] = 3*(county_NAICS4[
    'qtr'] - 1) + county_NAICS4['level_4']
county_NAICS4['date'] = pd.to_datetime(
    county_NAICS4[['year', 'mon']].astype(str).apply('-'.join,1))
county_NAICS4 = county_NAICS4.loc[county_NAICS4['date'] <= pd.Timestamp(2020, 3, 1)]
county_NAICS4 = county_NAICS4.drop(['qtr','level_4', 
                              'year', 'mon'], axis = 1)

# Employment level data by month
county_NAICS4 = county_NAICS4.rename({0:'emplvl', 
                                'industry_code' : 'NAICS4'}, axis = 1)    
county_NAICS4.tail(10)

,area_fips,NAICS4,emplvl,date
23595044,78030,8141,0,2019-06-01
23595045,78030,8141,0,2019-07-01
23595046,78030,8141,0,2019-08-01
23595047,78030,8141,0,2019-09-01
23595048,78030,8141,0,2019-10-01
23595049,78030,8141,0,2019-11-01
23595050,78030,8141,0,2019-12-01
23595051,78030,8141,0,2020-01-01
23595052,78030,8141,0,2020-02-01
23595053,78030,8141,0,2020-03-01


In [110]:
# Get the base level (2017) county industry level employment
# I know this looks ugly but is much faster than doing resample
# This block should be fast

# whole 2017
y2017 = []
for i in range(1, 13):
    y2017.append(pd.Timestamp(2017,i,1))

# Get the sum of Q1 empploy at county industry level
base_year = county_NAICS4.set_index('date').loc[
    y2017].groupby(['area_fips', 
                    'NAICS4']).sum()

# Now get the weights of each industry in US counties
base_year = base_year.reset_index()
temp = base_year.groupby('area_fips').sum().drop('NAICS4', axis = 1)

temp = temp.rename({'emplvl' : 'total_employment'}, axis = 1)

# Merge back with total base quarter employment level
base_year = pd.merge(base_year, temp,
                       how = "left",
                       left_on = 'area_fips',
                       right_on = 'area_fips')
base_year['weight'] = base_year['emplvl']/base_year['total_employment']
base_year = base_year.drop(['emplvl', 'total_employment'], axis = 1)
base_year.head(10)

,area_fips,NAICS4,weight
0,1001,1111,0.000000
1,1001,1114,0.000000
2,1001,1119,0.000000
3,1001,1121,0.000000
4,1001,1129,0.000000
5,1001,1132,0.000000
6,1001,1133,0.000000
7,1001,1151,0.000000
8,1001,1152,0.000000
9,1001,1153,0.004863


In [111]:
# Merge back
# This is also quick
county_NAICS4 = pd.merge(county_NAICS4, base_year,
                        how = "left",
                        left_on = ["area_fips", "NAICS4"],
                        right_on = ["area_fips", "NAICS4"])
county_NAICS4.tail()

,area_fips,NAICS4,emplvl,date,weight
23595049,78030,8141,0,2019-11-01,0.001317
23595050,78030,8141,0,2019-12-01,0.001317
23595051,78030,8141,0,2020-01-01,0.001317
23595052,78030,8141,0,2020-02-01,0.001317
23595053,78030,8141,0,2020-03-01,0.001317


In [112]:
# Merge with import tariff data
countytariff = pd.merge(county_NAICS4, NAICS4fullset[["NAICS4", "date", "weighted_tariff"]],
                        how = "left",
                        left_on = ["NAICS4","date"],
                        right_on = ["NAICS4", "date"])

In [113]:
# Calculated the tariff exposure at county level
countytariff['exposure'] = countytariff['weight']*countytariff['weighted_tariff']
countyexposure_import = countytariff.groupby(['area_fips', 'date'])['exposure'].sum().reset_index()
countyemp = county_NAICS4.drop(['weight', 'NAICS4'], axis = 1).groupby(['area_fips', 'date']).sum().reset_index()
countyexposure_import = pd.merge(countyexposure_import, countyemp,
                         how = "left",
                         left_on = ["area_fips", "date"],
                         right_on = ["area_fips", "date"])
countyexposure_import = countyexposure_import[countyexposure_import['date'] != pd.Timestamp(2020,3,1)]
countyexposure_import[countyexposure_import['area_fips'] == 5043].tail(20)

,area_fips,date,exposure,emplvl
6918,5043,2018-07-01,0.346436,1883
6919,5043,2018-08-01,0.346436,1847
6920,5043,2018-09-01,0.808388,1763
6921,5043,2018-10-01,2.287540,1385
6922,5043,2018-11-01,2.287540,1362
6923,5043,2018-12-01,2.287540,1380
6924,5043,2019-01-01,2.287540,1694
6925,5043,2019-02-01,2.287540,1678
6926,5043,2019-03-01,2.287540,1675
6927,5043,2019-04-01,2.287540,2262


In [114]:
# Exposure change of each county from Jan 2018 to Dec 2019
# That is, how much the import tariff exposure of each county change from before the trade war to before the first stage agreement
subsample = countyexposure_import[(countyexposure_import['date'] == '2018-01-01') |
                                  (countyexposure_import['date'] == '2019-12-01')]

# Shift to calculate difference in exposure
subsample['exposure2018Jan'] = subsample.groupby('area_fips')['exposure'].shift()

# Only keep shifted data to calculate exposure change
subsample = subsample[subsample['date'] == '2019-12-01']
subsample['exposurechange'] = subsample['exposure'] - subsample['exposure2018Jan']
subsample.head()

C:\anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,area_fips,date,exposure,emplvl,exposure2018Jan,exposurechange
47,1001,2019-12-01,0.343262,4662,0.017235,0.326026
98,1003,2019-12-01,1.076978,59534,0.109124,0.967854
149,1005,2019-12-01,1.017955,2297,0.276436,0.741519
200,1007,2019-12-01,2.299119,1022,0.000000,2.299119
251,1009,2019-12-01,1.508250,3566,0.077965,1.430284


In [115]:
# Assign groups to counties based on their total exposure change
subsample['group'] = 0
for i in subsample.index:
    if (subsample.loc[i]['exposurechange'] <= subsample['exposurechange'].quantile(0.25)) & (
    subsample.loc[i]['exposurechange'] > 0):
        subsample.loc[i, 'group'] = 1
    elif (subsample.loc[i]['exposurechange'] <= subsample['exposurechange'].quantile(0.5)) & (
    subsample.loc[i]['exposurechange'] > subsample['exposurechange'].quantile(0.25)):
        subsample.loc[i, 'group'] = 2
    elif (subsample.loc[i]['exposurechange'] <= subsample['exposurechange'].quantile(0.75)) & (
    subsample.loc[i]['exposurechange'] > subsample['exposurechange'].quantile(0.5)):
        subsample.loc[i, 'group'] = 3
    elif subsample.loc[i]['exposurechange'] > subsample['exposurechange'].quantile(0.75):
        subsample.loc[i, 'group'] = 4
subsample.head()

,area_fips,date,exposure,emplvl,exposure2018Jan,exposurechange,group
47,1001,2019-12-01,0.343262,4662,0.017235,0.326026,2
98,1003,2019-12-01,1.076978,59534,0.109124,0.967854,2
149,1005,2019-12-01,1.017955,2297,0.276436,0.741519,2
200,1007,2019-12-01,2.299119,1022,0.000000,2.299119,3
251,1009,2019-12-01,1.508250,3566,0.077965,1.430284,3


In [116]:
# merge back to monthly data, now we have exposure change and group at the monthly level data
countyexposure_import = pd.merge(countyexposure_import, 
                                 subsample[['area_fips', 'exposurechange', 'group']],
                                 how = "left",
                                 left_on = 'area_fips',
                                 right_on = 'area_fips')
countyexposure_import[countyexposure_import['area_fips'] == 5043].tail(20)

,area_fips,date,exposure,emplvl,exposurechange,group
6782,5043,2018-07-01,0.346436,1883,4.572745,4.0
6783,5043,2018-08-01,0.346436,1847,4.572745,4.0
6784,5043,2018-09-01,0.808388,1763,4.572745,4.0
6785,5043,2018-10-01,2.287540,1385,4.572745,4.0
6786,5043,2018-11-01,2.287540,1362,4.572745,4.0
6787,5043,2018-12-01,2.287540,1380,4.572745,4.0
6788,5043,2019-01-01,2.287540,1694,4.572745,4.0
6789,5043,2019-02-01,2.287540,1678,4.572745,4.0
6790,5043,2019-03-01,2.287540,1675,4.572745,4.0
6791,5043,2019-04-01,2.287540,2262,4.572745,4.0


In [117]:
countyexposure_import = countyexposure_import.dropna()
# Save for plotting in Python and regression analysis in Stata 
countyexposure_import.to_csv('outputdata\countyexposure_import.csv')